In [1]:
using CSV, DataFrames, Distances, DelimitedFiles, Plots

In [2]:
using HDF5, JLD

In [3]:
cities = CSV.read("cities_p.csv");

In [4]:
primes = findall(cities[:primes] .== true);

In [12]:
function get_clusters(cities)
    prime_path = convert.(Int,readdlm("prime_path"));
    used = zeros(Bool,size(cities)[1])
    used[1] = true
    used[prime_path] .= true
    pc = 1
    cluster = Dict{Int,Vector{Int}}()
    needed_extras = size(cities)[1]-10*length(primes)
    println("Extras: ", needed_extras)
    for p in prime_path[1:end-1]
        if (pc % 100 == 0 && pc <= 1000) || pc % 1000 == 0
            println("Pc: ", pc)
            println("Extras: ", needed_extras)
        end
        p1 = [cities[:X][p];cities[:Y][p]]
        unused = findall(used .== false)
        p2s = zeros(2, length(unused))
        p2s[1,:] = cities[:X][unused]
        p2s[2,:] = cities[:Y][unused]
        dist = Distances.colwise(SqEuclidean(), p1,p2s)
        if length(dist) >= 29
            psorted = partialsortperm(dist, 1:29)
        else
            psorted = collect(1:length(dist))
        end
        vals = 9
        if pc == 1
            vals = min(8,length(dist))
        elseif needed_extras > 0 && dist[psorted[1]] < 100
            vals = min(29,length(dist))
            needed_extras -= 20
        elseif needed_extras > 0 && dist[psorted[1]] < 400
            vals = min(19,length(dist))
            needed_extras -= 10
        else
            vals = min(9,length(dist))
        end 
        cluster[p] = unused[psorted[1:vals]]    
        used[cluster[p]] .= true
        pc += 1
    end
    return cluster, used
end
cluster, used = get_clusters(cities);

Extras: 19749
Pc: 100
Extras: 19419
Pc: 200
Extras: 19309
Pc: 300
Extras: 19059
Pc: 400
Extras: 18829
Pc: 500
Extras: 18819
Pc: 600
Extras: 18549
Pc: 700
Extras: 18469
Pc: 800
Extras: 18319
Pc: 900
Extras: 18039
Pc: 1000
Extras: 17899
Pc: 2000
Extras: 16239
Pc: 3000
Extras: 15339
Pc: 4000
Extras: 14259
Pc: 5000
Extras: 12569
Pc: 6000
Extras: 10929
Pc: 7000
Extras: 8669
Pc: 8000
Extras: 8439
Pc: 9000
Extras: 7309
Pc: 10000
Extras: 6879
Pc: 11000
Extras: 6609
Pc: 12000
Extras: 5209
Pc: 13000
Extras: 4219
Pc: 14000
Extras: 2519
Pc: 15000
Extras: 1709
Pc: 16000
Extras: 859
Pc: 17000
Extras: -1


In [13]:
println(length(used)-sum(used))

0


In [14]:
save("clusters.jld", "data", cluster)

In [8]:
[cities[:X][1:5],cities[:Y][1:5]]

2-element Array{Array{Union{Missing, Float64},1},1}:
 [316.837, 4377.41, 3454.16, 4688.1, 1010.7] 
 [2202.34, 336.602, 2820.05, 2935.9, 3236.75]

In [9]:
X = [0;0]
Y = [1 1; 2 2]'
println(Y)
Distances.colwise(Euclidean(), X, Y)

[1 2; 1 2]


2-element Array{Float64,1}:
 1.4142135623730951
 2.8284271247461903

In [10]:
[0 0; 0 0]

2×2 Array{Int64,2}:
 0  0
 0  0

In [11]:
@time euclidean([0,0],[1,1])
@time sqeuclidean([0,0],[1,1])

  0.039714 seconds (64.95 k allocations: 3.283 MiB, 36.76% gc time)
  0.017040 seconds (52.94 k allocations: 2.654 MiB)


2